In [22]:
#!/home/maghoi/.conda/envs/py37/bin/python
from Bio import SeqIO
import argparse, os, glob

#parser = argparse.ArgumentParser(description="Removes duplicate seqs from input file, writes to input_out")
#parser.add_argument("-i", dest="INPUT_FASTA", required=True, help="Input fasta file")
#parser.add_argument("-o", dest="OUTPUT_FASTA", help="Output fasta file")
#args = parser.parse_args()  
#dict_args = vars(args)


def de_duplicate_FASTA_files(inDir="../data/raw/", v=1):
    # Find train and test fasta files in raw
    fasta_files = glob.glob(inDir + "test*.fasta") + glob.glob(inDir + "train*.fasta")
    print("Checking for duplicate sequences in:\n", fasta_files, end="\n")
    outDir = "../data/interim/"

    # Count duplicates and entries across all fasta files
    dict_seqs = {}
    dict_ids = {}
    count_entries = 0

    # Load individual FASTA files and check whether sequences are duplicates before saving
    # IDs are only stored for counting
    for inFasta in fasta_files:
        outFasta = outDir + "filtered_" + os.path.basename(inFasta)

        with open(outFasta, 'w') as outHandle:
            no_duplicates = 0
            for record in SeqIO.parse(inFasta, "fasta"):
                count_entries += 1

                # Store unique sequences for later checking
                # Write ID+sequence if sequence is unique
                if str(record.seq) not in dict_seqs:
                    dict_seqs[str(record.seq)] = True
                    SeqIO.write(record, outHandle, 'fasta')

                    # Store unique IDs
                    if str(record.id) not in dict_ids:
                        dict_ids[str(record.id)] = True

                else:
                    no_duplicates += 1

            if v:
                print(f"Found {no_duplicates} duplicate sequences in {inFasta}")
                print(f"Saving to {outFasta}")
                
    # Print statistics
    print(f"\nProcessed {count_entries} sequences from {len(fasta_files)} train/test FASTA files in {inDir} to {outDir}:")
    print(f"Unique IDs / Sequences: {len(dict_ids.keys())} / {len(dict_seqs.keys())}")

de_duplicate_FASTA_files()

Checking for duplicate sequences in:
 ['../data/raw/test_neg.fasta', '../data/raw/test_pos.fasta', '../data/raw/train_pos.fasta', '../data/raw/train_neg.fasta']
Found 0 duplicate sequences in ../data/raw/test_neg.fasta
Saving to ../data/interim/filtered_test_neg.fasta
Found 0 duplicate sequences in ../data/raw/test_pos.fasta
Saving to ../data/interim/filtered_test_pos.fasta
Found 13 duplicate sequences in ../data/raw/train_pos.fasta
Saving to ../data/interim/filtered_train_pos.fasta
Found 256 duplicate sequences in ../data/raw/train_neg.fasta
Saving to ../data/interim/filtered_train_neg.fasta

Processed 3246 sequences from 4 train/test FASTA files in ../data/raw/ to ../data/interim/:
Unique IDs / Sequences: 2977 / 2977


In [24]:
call("" + " myarg", shell=True)

'nonamp5_30_1529'

In [ ]:
call("${PYTHON_INTERPRETER} -u " + "src/data/extract.py " + "", shell=True)

In [46]:
import subprocess
subprocess.run("echo hello world", shell=True, capture_output=True)

CompletedProcess(args='echo hello world', returncode=0, stdout=b'hello world\n', stderr=b'')

In [47]:
for fastaFile in glob.glob("../data/interim/test.fasta"):
    print(fastaFile)

    fastaName = Path(os.path.basename(fastaFile)).stem
    script = "${PYTHON_INTERPRETER} -u extract.py"
    model= "esm1_t6_43M_UR50S"
    params = "include per_tok"
    outDir = "../data/interim/" + fastaName + "/"

    cmd = " ".join([script, model, fastaFile, outDir, params])
    print(cmd)
    subprocess.run(cmd, shell=True, capture_output=True)

../data/interim/test.fasta
${PYTHON_INTERPRETER} -u extract.py esm1_t6_43M_UR50S ../data/interim/test.fasta ../data/interim/test/ include per_tok


In [53]:
import subprocess
with open('../src/data/run_esm.sh', 'rb') as file:
    script = file.read()
subprocess.run("bash ../src/data/run_esm.sh", shell=True)

CompletedProcess(args='bash ../src/data/run_esm.sh', returncode=2)

In [55]:
!pwd

/Users/maghoi/repos/__personal/MLOps_sequences/notebooks


In [58]:
!realpath bash ../src/data/run_esm.sh

/Users/maghoi/repos/__personal/MLOps_sequences/notebooks/bash
/Users/maghoi/repos/__personal/MLOps_sequences/src/data/run_esm.sh


In [63]:
script="../src/data/run_esm.sh"
os.chmod(script, 0o755)
subprocess.Popen(script)

<Popen: returncode: None args: '../src/data/run_esm.sh'>

In [64]:
!ls -alhtr ../src/data/

total 32
-rw-r--r--  1 maghoi  staff   5.0K Aug  5 12:49 extract.py
-rw-r--r--  1 maghoi  staff     0B Jan 12 16:12 .gitkeep
-rw-r--r--  1 maghoi  staff     0B Jan 12 16:12 __init__.py
drwxr-xr-x  8 maghoi  staff   256B Jan 12 17:55 ..
-rw-r--r--@ 1 maghoi  staff   3.1K Jan 12 18:13 make_dataset.py
drwxr-xr-x  7 maghoi  staff   224B Jan 12 18:23 .
-rwxr-xr-x  1 maghoi  staff   497B Jan 12 18:43 run_esm.sh
